## Demonstração do serviço de texto para fala (Speech to text) Amazon Polly


Aqui você verá um resumo das funcionalidades que a Polly possui. 
O texto de entrada do serviço pode ser puro ou no formato SSML. O formato SSML é uma marcação de texto padrão W3C. Esse padrão serve para descrever a forma com que um dado texto deve ser 'lido' pelo sintetizador. Com os exemplos abaixo ficará mais claro como usar esse formato.

Obs.: Todo o conteúdo está em Português.

Então vamos lá!

In [1]:
# Make sure you've installed boto3 version >=1.4.8
!conda upgrade --yes boto3

Fetching package metadata ...........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /home/ec2-user/anaconda3/envs/python2:
#
boto3                     1.5.32                   py27_0  


In [1]:
import boto3
import numpy as np
import base64
import os
import time

from io import BytesIO
from IPython.core.display import display, HTML, Javascript, display_javascript

In [2]:
err_message = "Try shutdown your kernel and start it again. It seems an old version was cached"
assert boto3.__version__ >= "1.4.8", err_message

Depois de importar as bibliotecas, vamos criar um cliente para a Polly e um mixer. O Mixer será utilizado para a reprodução do áudio retornado pela Polly.

In [3]:
client = boto3.client('polly')


In [4]:
html = """
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.2.1/jquery.min.js"></script>
<audio id="audio"><source id="media" src="audio.mp3" type="audio/mpeg" /></audio>
<script type="text/javascript">    
    function play_audio(){
        $("#audio")[0].load();
        $("#audio")[0].play();
    } // play_audio
</script>
"""
HTML(html)

Agora vamos definir algumas funções auxiliares para:
- tocar um streaming de áudio
- sintetizar a fala a partir de um arquivo SSML
- carregar um arquivo do disco

In [5]:
def play_audio(audiostream):
    audio = open('audio.mp3', 'wb+')
    audio.write(audiostream.read())
    audio.flush()
    audio.close()
    
    jso = Javascript('play_audio()')
    display_javascript(jso)

In [6]:
def sintetiza(ssml, lexicons=[], voiceid='Ricardo' ):
    if os.path.isfile('audio.mp3'):
        os.remove('audio.mp3') 
    response = client.synthesize_speech(
        LexiconNames=lexicons,
        OutputFormat='mp3',
        SampleRate='22050',
        Text=ssml,
        TextType='ssml',
        VoiceId=voiceid
    )
    play_audio( response['AudioStream'] )
    print(ssml)

In [7]:
def load_file( filename ):
    return "".join( open( filename, 'r').readlines() )

Ok. Agora vamos fazer um breve teste de leitura. Note as tags &lt;speak&gt; e &lt;p&gt;. Elas fazem parte do padrão SSML.

In [8]:
sintetiza( load_file( 'poema.ssml' ) )

<speak>

<p>O Mundo não se Fez para Pensarmos Nele, por Alberto Caeiro</p>

<p>O meu olhar é nítido como um girassol.
Tenho o costume de andar pelas estradas
Olhando para a direita e para a esquerda,
E de, vez em quando olhando para trás...</p>

<p>E o que vejo a cada momento
É aquilo que nunca antes eu tinha visto,
E eu sei dar por isso muito bem...</p>

</speak>



### Tags SSML suportadas pela Polly
http://docs.aws.amazon.com/polly/latest/dg/supported-ssml.html
 - &lt;break&gt;: Intervalo em seg ou mili
 - &lt;lang&gt;: Corrige pronúncia de um estrangeirismo
 - &lt;mark&gt;: Adiciona marcações no texto, sem alterar a reprodução
 - &lt;p&gt;: Configura um novo parágrafo com uma pausa.
 - &lt;phoneme&gt;: Interpreta fonemas
 - &lt;prosody&gt;: Regula a cadência da fala: velocidade, entonação, etc
 - &lt;s&gt;: Dá uma pausa após ler a sentença
 - &lt;say-as&gt;: Muda a forma de se ler um texto: hora, telefone, etc
 - &lt;sub&gt;: Substitui um termo por uma sentença
 - &lt;w&gt;: Altera a pronúncia de uma palavra
 - &lt;amazon:effect name="whispered"&gt;: Sussurrar
 - &lt;amazon:effect name="drc"&gt;: Realçar som




### P & Lang

In [9]:
sintetiza( load_file( 'lang.ssml' ) )

<speak>
<p>Comprei um notebook novo</p>
<p>Comprei um <lang xml:lang="en-US">notebook</lang> novo</p>
</speak>



## Break

In [10]:
sintetiza( load_file( 'break.ssml' ) )

<speak>
<p>Com grandes poderes vêm grandes responsabilidades</p>
<p>Com grandes poderes<break strength="medium"/> vêm grandes responsabilidades</p>
<p>Com grandes poderes<break time="2s"/> vêm grandes responsabilidades</p>
</speak>



## Say as

In [11]:
sintetiza( load_file( 'sayas.ssml' ) )

<speak>
<s>O celular do presidente é: 9816-0245</s>
O celular do presidente é: <say-as interpret-as="telephone">9816-0245</say-as>
</speak>



## Lexicons

#### Carrega o arquivo de lexicons

In [12]:
acronimos = load_file( 'lexicons.pls' )
print(acronimos)
response = client.put_lexicon(
    Name='acronimos',
    Content=acronimos
)

<?xml version="1.0" encoding="UTF-8"?>
<lexicon version="1.0" 
      xmlns="http://www.w3.org/2005/01/pronunciation-lexicon"
      xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" 
      xsi:schemaLocation="http://www.w3.org/2005/01/pronunciation-lexicon 
        http://www.w3.org/TR/2007/CR-pronunciation-lexicon-20071212/pls.xsd"
      alphabet="ipa" 
      xml:lang="pt-BR">
  <lexeme>
    <grapheme>RFFSA</grapheme>
    <alias>Rede Ferroviária Federal Sociedade Anônima</alias>
  </lexeme>
  <lexeme>
    <grapheme>FENPROF</grapheme>
    <alias>Federação Nacional de Professores</alias>
  </lexeme>
  <lexeme>
    <grapheme>H2O</grapheme>
    <alias>água</alias>
  </lexeme>
  <lexeme>
    <grapheme>H_</grapheme>
    <alias>Hidrogênio</alias>
  </lexeme>
  <lexeme>
    <grapheme>O_</grapheme>
    <alias>Oxigênio</alias>
  </lexeme>
</lexicon>



In [13]:
sotaques = {
    'uai': 'uai.pls',
    'tche': 'tche.pls',
    'meu': 'meu.pls'
}
for nome, nome_arquivo in sotaques.items():
    response = client.put_lexicon(
        Name=nome,
        Content=load_file( nome_arquivo )
    )

In [14]:
sintetiza( load_file( 'lexicon.ssml' ) )

<speak>
<p>O meu avô trabalhou durante 40 anos na RFFSA</p>
</speak>



In [15]:
sintetiza( load_file( 'lexicon.ssml' ), [ 'acronimos'] )

<speak>
<p>O meu avô trabalhou durante 40 anos na RFFSA</p>
</speak>



In [16]:
sintetiza( load_file( 'agua.ssml' ), [ 'acronimos'] )

<speak>
<p>A H2O é composta por dois átomos de H_ e um átomo de O_</p>
</speak>



#### Mineirês

In [17]:
sintetiza( load_file( 'lexicon_sotaques.ssml' ), [ 'uai'] )

<speak>
<p>Isso aqui é muito simples []!</p>
</speak>



#### Paulistês

In [18]:
sintetiza( load_file( 'lexicon_sotaques.ssml' ), [ 'meu'] )

<speak>
<p>Isso aqui é muito simples []!</p>
</speak>



#### Gauchês

In [19]:
sintetiza( load_file( 'lexicon_sotaques.ssml' ), [ 'tche'] )

<speak>
<p>Isso aqui é muito simples []!</p>
</speak>



### Sussurrar & gritar

In [20]:
sintetiza( load_file( 'alteracao_voz.ssml' ) )

<speak>
Então ela entrou no quarto correndo e resmungou: <amazon:effect name="drc"><prosody volume="x-loud" rate="x-slow" pitch="x-high">Apolônio! Apolônio!</prosody></amazon:effect>.
Ele respondeu: <amazon:effect name="drc"><prosody volume="loud" rate="fast" pitch="x-high">o quê!</prosody></amazon:effect>
E ela sussurrou em seu ouvido: <amazon:effect name="whispered">Com grandes poderes vêm grandes responsabilidades</amazon:effect>
</speak>



### Breath

In [27]:
ssml = """
<speak>
     Um ninho de mafagafos tem cinco mafagafinhos.<amazon:breath duration="short" volume="x-loud"/>
     Quem desmafagafizar os mafagafos bom desmafagafizador será.
</speak>
"""
sintetiza(ssml, voiceid='Ricardo')


<speak>
     Um ninho de mafagafos tem cinco mafagafinhos.<amazon:breath duration="short" volume="x-loud"/>
     Quem desmafagafizar os mafagafos bom desmafagafizador será.
</speak>



In [28]:
ssml = """
<speak>
   <amazon:auto-breaths>
Um Anel para a todos governar, Um Anel para encontrá-los, 
Um Anel para a todos trazer e na escuridão aprisioná-los. 
Na Terra de Mordor onde as Sombras se deitam." 
Por causa desse poema, muitas vezes os Anéis são referidos apenas como os Três, os Sete e os Nove, 
assim como o Anel de Sauron, chamado de o Um.
   </amazon:auto-breaths>
</speak>
"""
sintetiza(ssml, voiceid='Ricardo')


<speak>
   <amazon:auto-breaths>
Um Anel para a todos governar, Um Anel para encontrá-los, 
Um Anel para a todos trazer e na escuridão aprisioná-los. 
Na Terra de Mordor onde as Sombras se deitam." 
Por causa desse poema, muitas vezes os Anéis são referidos apenas como os Três, os Sete e os Nove, 
assim como o Anel de Sauron, chamado de o Um.
   </amazon:auto-breaths>
</speak>



# Hack the following code and use the latest feature (Timbre)
http://docs.aws.amazon.com/polly/latest/dg/supported-ssml.html#vocaltractlength-tag


In [21]:
# English voices: F(Joanna Salli Kimberly Kendra Ivy Matthew) M(Justin Joey)
# Spanish voices: F(Penelope) M(Miguel)
# Portuguese voices: F(Vitoria) M(Ricardo)
ssml = """
<speak>
 your text  <amazon:effect vocal-tract-length="+15%"> with the effect effect effect </amazon:effect> here
</speak>
"""
sintetiza(ssml, voiceid='Ivy')


<speak>
 your text  <amazon:effect vocal-tract-length="+15%"> with the effect effect effect </amazon:effect> here
</speak>

